'c:\\Users\\tashi\\Programming\\arXiv_bot'

In [12]:
# 使用するライブラリのインポート
import arxiv
import pandas as pd
import os

def fetch_papers_from_arxiv(search_query=None, max_results=3, csv_filename = "datas/arxiv_papers.csv") -> pd.DataFrame:
    """
    arXiv APIを使用して論文を取得し、CSVファイルに保存する関数

    Parameters:
        search_query (dict{"thema": "query"}): 検索クエリ
            ex) queries = {
                    "LLM": 'cat:cs.CL OR cat:cs.AI AND "large language model"',
                    "Machine Learning": 'cat:cs.LG OR cat:stat.ML AND "machine learning"',
                    "XAI": 'cat:cs.AI OR cat:cs.LG AND "explainable AI"'
                }
        max_results (int): 取得する論文の最大数
        csv_filename (str): 保存するCSVファイル名

    Returns:
        DataFrame: 新しい論文情報を含むDataFrame
    """
    if search_query is None:
        # テーマごとのクエリ
        queries = {
            "LLM": 'cat:cs.CL OR cat:cs.AI AND "large language model"',
            "Machine Learning": 'cat:cs.LG OR cat:stat.ML AND "machine learning"',
            "XAI": 'cat:cs.AI OR cat:cs.LG AND "explainable AI"'
        }
    
    # 新しく取得した論文の情報を格納するリスト
    new_papers_list = []

    # 既存のCSVファイルを読み込む（存在すれば）
    if os.path.exists(csv_filename):
        existing_papers_df = pd.read_csv(csv_filename)
    else:
        # CSVが存在しない場合は空のDataFrameを作成
        existing_papers_df = pd.DataFrame(columns=["Title", "Authors", "Published", "URL", "Abstract", "PDF"])

    # 既存のURL一覧を取得
    existing_urls = existing_papers_df["URL"].tolist()

    # 各クエリで論文を取得し、既存のデータと比較
    for theme, query in queries.items():
        search = arxiv.Search(
            query=query,
            max_results=max_results,  # 各テーマで取得する最大論文数
            sort_by=arxiv.SortCriterion.SubmittedDate
        )

        for result in search.results():
            # 論文情報を辞書にまとめる
            paper_info = {
                "Title": result.title,
                "Theme": theme,
                "Authors": ', '.join([author.name for author in result.authors]),
                "Published": str(result.published),
                "URL": result.entry_id,
                "Abstract": result.summary,
                "PDF": result.pdf_url
            }

            # 既存の論文リストにないか、または更新されている場合はリストに追加
            if paper_info["URL"] not in existing_urls or paper_info["Published"] != existing_papers_df.loc[existing_papers_df["URL"] == paper_info["URL"], "Published"].values[0]:
                new_papers_list.append(paper_info)

    # 新しい論文があれば処理
    if new_papers_list:
        # 新しい論文のDataFrameを作成
        new_papers_df = pd.DataFrame(new_papers_list)

        # 既存の論文と新しい論文を結合
        updated_papers_df = pd.concat([existing_papers_df, new_papers_df], ignore_index=True)

        # CSVファイルに上書き保存
        updated_papers_df.to_csv(csv_filename, index=False)

        # print(f"{len(new_papers_list)} 件の新しい論文を追加しました。")

    return new_papers_df


In [13]:
# from ..libs.fetch_arxiv import fetch_papers_from_arxiv
# from libs.summarize import get_summary_by_gpt
# from libs.to_message import send_message_to_discord

# def main():
#     # arXiv APIを使用して論文を取得し、CSVファイルに保存
#     df_papers = fetch_papers_from_arxiv()

In [14]:
df_papers = fetch_papers_from_arxiv()

C:\Users\tashi\AppData\Local\Temp\ipykernel_29716\76235368.py:52: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


In [16]:
import pandas as pd
from openai import OpenAI
import pandas as pd

def get_summary_by_gpt(api_key: str, df_papers: pd.DataFrame) -> dict:
    """論文の内容を要約したものを返す

    Args:
        api_key (str): OpenAI API キー
        df_papers (pd.DataFrame): 論文データ

    Returns:
        dict: レスポンスデータ（jsonをdictに変換したもの）
    """

    client = OpenAI(api_key=api_key)

    # リクエストのプロンプトを作成する
    prompt = "論文の概要を日本語でそれぞれ要約して。要約文のみ出力。"
    prompt += "出力例：pythonのlist形式。"
    prompt += str(df_papers["Abstract"][:2].to_list())

    # OpenAI API にリクエストを送信する
    completion = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=[
            {"role": "system", "content": "読者の興味を引く要素を強調"}, # 何かを知るにはまず興味をそそられることが重要
            {"role": "user", "content": prompt}
        ]
    )

    response = completion.choices[0].message.to_dict()

    return response

In [19]:
import json
key = json.load(open("openai_apikey.json"))["KEY"]

res = get_summary_by_gpt(api_key=key, df_papers=df_papers)

In [21]:
# ここにSlackなどのメッセージ送信用の関数を記述する

from discord_webhook import DiscordWebhook

def send_message_to_discord(message: str, webhook_url: str):
    """Discordにメッセージを送信する

    Args:
        message (str): 送信するメッセージ
        webhook_url (str): DiscordのWebhook URL

    Returns:
        response: webhookによるレスポンス
    """
    # メッセージを送信
    webhook = DiscordWebhook(url=webhook_url, content=message)
    response = webhook.execute()

    return response

In [26]:
from ast import literal_eval

webhookurl = json.load(open("sample_code\webhook_url.json"))["URL"]
papers_summary = literal_eval(res["content"])

# send_message_to_discord()

['Chain-of-thought (CoT)プロンプトは、大規模言語モデル(LLM)から推論能力を引き出すための実質的な方法ですが、どのようなタスクでこの「思考」が本当に有益なのかを分析するために、100以上の論文を対象とした定量的メタ分析を実施し、14モデルで20のデータセットを評価しました。その結果、CoTは数学や論理に関するタスクで強い性能向上を提供し、他のタスクでは効果が小さいことがわかりました。また、MMLUでは、CoTを使用しなくてもほぼ同じ精度が得られることが示されました。この結果を受けて、計画と実行を分けてCoTの挙動を分析し、ツール補強されたLLMと比較しました。CoTの多くの利点は象徴的な実行の改善から来ており、象徴的ソルバーを使用することと比較すると劣ります。CoTは選択的に適用され、推論コストを節約しつつ性能を維持できることを示唆しています。',
 '個別化されたアウトフィット推薦は、ファッションの互換性とトレンドの理解を必要とする複雑な課題です。本論文では、大規模言語モデル（LLM）の表現力を活かした新しいフレームワークを提案し、LLMの「ブラックボックス」性と静的な特性をファインチューニングと直接フィードバックの統合によって緩和します。視覚テキストのギャップを埋めるために、マルチモーダル大規模言語モデル（MLLM）を用いた画像キャプショニングを活用し、ファッション画像からスタイルやカラーの特徴を抽出します。LLMはオープンソースのPolyvoreデータセットで効率的にファインチューニングされ、スタイリッシュなアウトフィットを推奨する能力を最適化します。ネガティブ例を使用した直接的な嗜好メカニズムが、LLMの意思決定プロセスを向上させるために採用され、季節のファショントレンドに合致したアウトフィット提案を継続的に洗練する自己強化型AIフィードバックループが形成されます。このフレームワークはPolyvoreデータセットで評価され、空白埋めや補完アイテム探索の2つの主要タスクでその効果が示されました。評価結果により、提案したフレームワークがベースLLMを大幅に上回り、より統一感のあるアウトフィットを生成できることが示されました。このタスクにおける性能向上は、正確な提案でショッピング体験を向上させるポテンシャルを強調しています。']